In [1]:
%cd ../

/Users/adam/Dropbox/CanvasCorrections


In [2]:

from ipywidgets import widgets
from IPython.display import display


In [ ]:
import environment

In [ ]:
environment.CONFIG

In [ ]:
environment.TOKEN

# Setup the configuration

## Canvas API Token

Instructions go here....

In [3]:
from assets.Configuration import InteractiveConfiguration
from assets.InputFields import make_course_ids_input, make_canvas_token_input, make_general_reset_button

In [ ]:
make_canvas_token_input()

In [ ]:
# You may confirm that the token has been added to the configuration here
InteractiveConfiguration.canvas_token

## Course ids

Instructions go here

In [11]:
make_course_ids_input()

In [ ]:
sections = [67531]

# Figure out what needs grading

In [4]:
from assets.RequestTools import get_all_course_assignments, get_assignments_needing_grading

Reading credentials and settings from /Users/adam/Dropbox/CanvasCorrections/private/canvas-credentials.ini


In [ ]:
print("These assignments need grading: ")
for sec in InteractiveConfiguration.course_ids:
    to_grade = get_assignments_needing_grading(sec)
    print(sec, to_grade)

In [9]:
to_grade = [('Journal (week 1)', 360442)]

In [ ]:
class DataStore(object):
    """All downloaded and score data gets stored in this object
    Methods using the data will expect an instance.
    """
    
    def __init__(self):
        self._credit = []
        self._nocredit = []
        self._submissions = []

# Download and process student submissions

Todo

    - Do we want to have more control over which assignments are graded? This just does all ungraded...

In [5]:
import assets.GradingTools as GT
import assets.DownloadProcessingTools as PT
from assets.DataManagement import DataStore

In [12]:
results = []

credit = []
nocredit = []

for course_id in InteractiveConfiguration.course_ids:
    print('course', course_id)
#     to_grade = get_assignments_needing_grading(course_id)
    
    for name, assignment_id in to_grade:
        store = DataStore(assignment_id=assignment_id, assignment_name=name, course_id=course_id)

        print("Processing {}".format(name))
        # make folder
#         folder = journal_folder_name(name, course_id)
#         create_folder(folder)
        # download student submissions 
        response = PT.get_submissions(course_id, assignment_id)
        print("{} responses received".format(len(response)))
        store.submissions = PT.process_response_without_saving_files(response)
        # save a copy 
#         PT.save_submission_json(submissions, folder)
        # give credit for non-empty submissions
        c = GT.determine_credit(store.submissions)
        # store the decisions rather than sending to server
        # that way we don't keep updating until ready
        store.credit = c['credit']
        store.no_credit = c['nocredit']
        store.print_counts()
        results.append(store)

#         store.credit.append({'course' : course_id, 'assignment': assignment_id, 'ids': c['credit']})
#         store.nocredit.append({'course' : course_id, 'assignment': assignment_id, 'ids': c['nocredit']})
#         print("Tentatively assigning credit to {} submissions; no credit to {} submissions.".format(len(store.credit), len(store.no_credit))


course 67531
Processing Journal (week 1)
https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions?per_page=42
https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions?page=2&per_page=42
https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions?page=3&per_page=42
126 responses received
Tentatively assigning credit to 87 submissions; no credit to 39 submissions.


# Read student work and check that properly categorized

In [17]:
def make_assignment_header(store):
    """Displays the header for each assignment"""
    entry = """
    <h1>{}</h1>
    <h2>Class id: {}</h2>"""
    return display(widgets.HTML(entry.format(store.assignment_name, store.course_id)))


def make_text_display(student_id, text):
    """Handles the formatting of the student's text"""
    entry = """
          <div id='%s'>
            <h3>%s</h3>
            <p>
                %s
            </p>
        </div>"""
    e = entry % (student_id, student_id, text)
    return widgets.HTML(e)

def make_submission_output(text, student_id, credit_list):
    """Creates the display of the submitted text with a toggle button to 
    update whether the student receives credit 
    """
    bval = 'Credit' if student_id in credit_list else 'No credit'
    credit_button = widgets.ToggleButtons(
        options=['Credit', 'No credit'],
        value=bval,
        description='',
        disabled=False
    )
    
    ta = make_text_display(student_id, text)

    def on_value_change(change):
        """The relevant callback
        NB the use of scope to define student_id"""
        v = change['new']
        print(v, student_id)
        try:
            if v == 'Credit':
                credit_list.append(student_id)
            elif v == 'No credit':
                credit_list.remove(student_id)
        except ValueError:
            pass

    credit_button.observe(on_value_change, names='value')
    display(widgets.VBox([ta, credit_button]))

def make_consolidated_text_fields(store):
    """Displays each entry with a toggle to adjust whether the 
    student receives credit"""
    for r in store.submissions:
        make_submission_output(r['body'], r['student_id'], store.credit)

In [18]:
test_results = [{'student_id' : i, 'body': 'Body text for student {}'.format(i)} for i in range(0,4)]

# credit_list = [0, 2]

# make_consolidated_text_fields(test_results, credit_list)

In [20]:
for s in results:
    make_assignment_header(s)
    make_consolidated_text_fields(s)

HTML(value='\n    <h1>Journal (week 1)</h1>\n    <h2>Class id: 67531</h2>')

# Upload the grades to canvas

In [38]:


def make_upload_button(store):
    def upload_callback(event):
        print(store.credit)
        # change button style
        b.button_style='success'
        
    desc = 'Upload grades for {}'.format(store.assignment_name)
    layout= widgets.Layout(width='50%')
    b = widgets.Button(description=desc, button_style='danger', layout=layout)
    b.on_click(upload_callback)
    
    display(b)

In [40]:
for store in results:
    make_upload_button(store)

Button(button_style='danger', description='Upload grades for Journal (week 1)', layout=Layout(width='50%'), st…